# Welcome to My Capstone Project Page!

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import geocoder
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# obtain Toronto info

In [3]:
CLIENT_ID = '3ZXMWIQUJV4T0PEWKY1JJRAVVBPA1NY5OL3G3HBNJGRY4HWT' # your Foursquare ID
CLIENT_SECRET = 'VF5OJMMW03GHLSIDNPWBYUPNCNEKB55GRLIYUASE0D5LZPUP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [4]:
LIMIT=200

def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# neighborhoods in Toronto with coordinates were put into a dataframe

In [5]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    trial_count=0
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        trial_count+=1
        if trial_count>0:
            idx=geospatial_coordinates[geospatial_coordinates['Postal Code']==postal_code].index[0]
            lat_lng_coords=(geospatial_coordinates.iloc[idx]['Latitude'],geospatial_coordinates.iloc[idx]['Longitude'])

    return lat_lng_coords

quote_page='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(quote_page).text

soup = BeautifulSoup(page, 'html.parser')
table = soup.find(lambda tag: tag.name=='table') 
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [value.text.strip('\n') for value in td]
    l.append(row)
l=l[1:]

df=pd.DataFrame(l, columns=["PostCode", "Borough", "Neighborhood"])
df.drop(df[(df['Borough']=='Not assigned') | (df['Borough']==None)].index,inplace=True)
neighborhood_not_assigned=df[df['Neighborhood']=='Not assigned']
df.loc[neighborhood_not_assigned.index,'Neighborhood']=neighborhood_not_assigned['Borough']
df = df.groupby('PostCode').agg({'Borough':'first', 
                             'Neighborhood': ', '.join}).reset_index()
geospatial_coordinates=pd.read_csv('Geospatial_Coordinates.csv')

col_lat=[]
col_lng=[]

for row in df.iterrows():
    postal_code=row[1]['PostCode']
    lat,lng=get_latlng(postal_code)
    col_lat.append(lat)
    col_lng.append(lng)

df['Latitude']=col_lat
df['Longitude']=col_lng
df.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# visualize those places on the map

In [6]:
neighborhoods=df
latitude,longitude=43.68, -79.39
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# venue information in those neighborhoods is extracted

In [7]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

In [8]:
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 332 uniques categories.


In [9]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
toronto_onehot.shape

(3969, 332)

In [11]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# most common venues in each neighborhoods

In [13]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,American Restaurant,Thai Restaurant,Theater,Bar,Concert Hall,Steakhouse,Gastropub
1,Agincourt,Chinese Restaurant,Shopping Mall,Shanghai Restaurant,Motorcycle Shop,Supermarket,Sandwich Place,Discount Store,American Restaurant,Lounge,Malay Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Fast Food Restaurant,Pizza Place,Park,Chinese Restaurant,BBQ Joint,Fried Chicken Joint,Coffee Shop,Pharmacy,Caribbean Restaurant,Noodle House
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Pharmacy,Sandwich Place,Liquor Store,Beer Store,Fried Chicken Joint,Coffee Shop,Fast Food Restaurant,Hardware Store
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gas Station,Pub,Pharmacy,Park,Convenience Store,Sandwich Place,Dance Studio,Skating Rink


# run K-means algorithm to identify clusters of neighborhoods in Toronto area

In [14]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=np.random.randint(1000)).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 1, 1, 1, 1, 4, 4, 4, 0])

In [15]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Spa,Martial Arts Dojo,Paper / Office Supplies Store,Hobby Shop,African Restaurant,Community Center,Dog Run
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4.0,Breakfast Spot,Italian Restaurant,Burger Joint,Bar,Women's Store,Electronics Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Pizza Place,Fast Food Restaurant,Coffee Shop,Plaza,Fried Chicken Joint,Beer Store,Pharmacy,Greek Restaurant,Sports Bar,Supermarket
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Park,Business Service,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Coffee Shop,Bakery,Indian Restaurant,Yoga Studio,Rental Car Location,Burger Joint,Bus Line,Flower Shop,Music Store,Lounge


In [16]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(-1)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Spa,Martial Arts Dojo,Paper / Office Supplies Store,Hobby Shop,African Restaurant,Community Center,Dog Run
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4,Breakfast Spot,Italian Restaurant,Burger Joint,Bar,Women's Store,Electronics Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Pizza Place,Fast Food Restaurant,Coffee Shop,Plaza,Fried Chicken Joint,Beer Store,Pharmacy,Greek Restaurant,Sports Bar,Supermarket
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Park,Business Service,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Coffee Shop,Bakery,Indian Restaurant,Yoga Studio,Rental Car Location,Burger Joint,Bus Line,Flower Shop,Music Store,Lounge


# visualize clustering results

In [17]:


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# begin input an address and identify 5 most similar places in Toronto

## input an address and find its coordinates

In [18]:
address = '14 N Bennet St, Boston, MA 02113' #Little iItaly, Boston

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

42.365053 -71.053983


## visualize the proposed place

In [19]:
map_query = folium.Map(location=[latitude, longitude], zoom_start=16)
    
map_query

## get venue information of the proposed place

In [20]:
query_venues = getNearbyVenues(names=['Place Query'],
                                   latitudes=[latitude],
                                   longitudes=[longitude]
                                  )

Place Query


In [21]:
query_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Place Query,42.365053,-71.053983,Locale,42.365055,-71.053271,Pizza Place
1,Place Query,42.365053,-71.053983,The Old North Church,42.366256,-71.054386,Church
2,Place Query,42.365053,-71.053983,Monica's Mercato,42.365077,-71.055444,Market
3,Place Query,42.365053,-71.053983,Prezza,42.364711,-71.052687,Italian Restaurant
4,Place Query,42.365053,-71.053983,Little Italy,42.363697,-71.054672,Neighborhood


In [22]:
query_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Place Query,100,100,100,100,100,100


In [23]:
print('There are {} uniques categories.'.format(len(query_venues['Venue Category'].unique())))

There are 40 uniques categories.


In [24]:
query_onehot=pd.DataFrame(data=np.zeros([query_venues.shape[0],toronto_grouped.shape[1]]), columns=toronto_grouped.columns)
for i in range(len(query_venues)):
    cat=query_venues.iloc[i]['Venue Category']
    if cat in query_onehot.columns and cat!='Neighborhood':
        query_onehot.iloc[i][cat]=query_onehot.iloc[i][cat]+1
query_onehot.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
query_onehot.shape

(100, 332)

In [26]:
query_grouped = query_onehot.groupby('Neighborhood').mean().reset_index()
query_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0


## find the most common venues of the proposed place

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = query_grouped['Neighborhood']

for ind in np.arange(query_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(query_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.0,Italian Restaurant,Bakery,Pizza Place,Seafood Restaurant,Park,Café,Market,Sandwich Place,Wine Shop,Historic Site


## predict the cluster of the proposed place

In [28]:
assigned_k=kmeans.predict(query_grouped.drop('Neighborhood', 1))[0]
assigned_k

4

## begin to find most similar places of the proposed place in Toronto

In [29]:
toronto_grouped_copy=toronto_grouped.copy()
toronto_grouped_copy

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
1,Agincourt,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
4,"Alderwood, Long Branch",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.047619,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
6,Bayview Village,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.024390,0.000000,0.000000,0.0,0.000000,0.0,0.024390,0.000000
8,Berczy Park,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000


In [30]:
toronto_grouped_copy['Distance to Query']=np.sqrt(((np.array(toronto_grouped_copy.iloc[:,1:])-np.array(query_grouped.iloc[:,1:]))**2).sum(1))
toronto_grouped_copy.sort_values('Distance to Query',inplace=True)
toronto_grouped_copy

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Distance to Query
51,"High Park, The Junction South",0.014706,0.000000,0.00,0.00,0.000000,0.000,0.0,0.0,0.0,...,0.00,0.014706,0.014706,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.360482
28,Davisville,0.021978,0.000000,0.00,0.00,0.000000,0.000,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.010989,0.000000,0.361141
7,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.00,0.00,0.000000,0.000,0.0,0.0,0.0,...,0.00,0.024390,0.000000,0.0,0.00,0.000000,0.0,0.024390,0.000000,0.364201
29,Davisville North,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.0,0.00,0.018182,0.0,0.000000,0.000000,0.370295
30,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.016129,0.000000,0.00,0.00,0.000000,0.000,0.0,0.0,0.0,...,0.00,0.000000,0.016129,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.371802
85,"The Annex, North Midtown, Yorkville",0.012346,0.000000,0.00,0.00,0.000000,0.000,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.373194
45,"Forest Hill North, Forest Hill West",0.000000,0.000000,0.00,0.00,0.000000,0.000,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.375566
90,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.000000,0.00,0.00,0.000000,0.000,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.376431
82,St. James Town,0.010000,0.000000,0.00,0.00,0.000000,0.000,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.381182
70,North Toronto West,0.032258,0.000000,0.00,0.00,0.000000,0.000,0.0,0.0,0.0,...,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.383824


## use Euclidean distance measure to identify the most similar places and list their most common venues

In [31]:
top_n=5
indices=[]
for i in range(top_n):
    indices.append(toronto_merged[toronto_merged['Neighborhood']==toronto_grouped_copy.iloc[i]['Neighborhood']].index[0])

most_similar_places=toronto_merged.iloc[indices]
most_similar_places

,PostCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,4,Café,Bar,Italian Restaurant,Bakery,Antique Shop,Coffee Shop,Deli / Bodega,Park,Mexican Restaurant,Pizza Place
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,4,Coffee Shop,Italian Restaurant,Sushi Restaurant,Sandwich Place,Café,Dessert Shop,Restaurant,Pizza Place,Fast Food Restaurant,Indian Restaurant
62,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,4,Italian Restaurant,Coffee Shop,Sushi Restaurant,Juice Bar,Fast Food Restaurant,Skating Rink,Baby Store,Bagel Shop,Bakery,Bank
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,4,Pizza Place,Café,Coffee Shop,Italian Restaurant,Park,Food & Drink Shop,Sporting Goods Shop,Fast Food Restaurant,Sushi Restaurant,Burger Joint
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,4,Coffee Shop,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Café,Gym,Spa,Sandwich Place,Pub,Pizza Place


## print most common venues of the proposed place for comparison purpose

In [32]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.0,Italian Restaurant,Bakery,Pizza Place,Seafood Restaurant,Park,Café,Market,Sandwich Place,Wine Shop,Historic Site


## visualize the 5 places found on the map, enjoy your exploration in Toronto!

In [33]:
# create map
latitude,longitude=43.68, -79.39
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(most_similar_places['Latitude'], most_similar_places['Longitude'], most_similar_places['Neighborhood'], most_similar_places['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.Marker(
        [lat, lon],
        #radius=5,
        popup=label,
        #color='red',
        #fill=True,
        #fill_color='red',
        #fill_opacity=0.7
    ).add_to(map_clusters)
       
map_clusters

In [36]:
toronto_merged[toronto_merged['Cluster Labels']==4]

,PostCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4,Breakfast Spot,Italian Restaurant,Burger Joint,Bar,Women's Store,Electronics Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Coffee Shop,Bakery,Indian Restaurant,Yoga Studio,Rental Car Location,Burger Joint,Bus Line,Flower Shop,Music Store,Lounge
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,4,Coffee Shop,Discount Store,Chinese Restaurant,Grocery Store,Convenience Store,Department Store,Intersection,Light Rail Station,Bank,Rental Car Location
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,4,Diner,Bus Line,Coffee Shop,Bakery,Convenience Store,Bus Station,Intersection,Park,Fast Food Restaurant,Soccer Field
10,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,4,Furniture / Home Store,Electronics Store,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Wings Joint,Coffee Shop,Vietnamese Restaurant,Latin American Restaurant,Gym / Fitness Center
11,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849,4,Bar,Restaurant,Seafood Restaurant,Rental Car Location,Furniture / Home Store,Fish Market,Korean Restaurant,Burger Joint,Convenience Store,Smoke Shop
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,4,Chinese Restaurant,Shopping Mall,Shanghai Restaurant,Motorcycle Shop,Supermarket,Sandwich Place,Discount Store,American Restaurant,Lounge,Malay Restaurant
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,4,Clothing Store,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Bank,Shopping Mall,Restaurant,Supermarket,Food Court,Movie Theater
19,M2K,North York,Bayview Village,43.786947,-79.385975,4,Japanese Restaurant,Bank,Skate Park,Café,Skating Rink,Grocery Store,Convenience Store,Chinese Restaurant,Restaurant,Shopping Mall
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,4,Korean Restaurant,Coffee Shop,Trail,Park,Café,Sandwich Place,Shopping Mall,Electronics Store,Ski Chalet,Middle Eastern Restaurant
